# Lab 5

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [1]:
import random
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = self.get_value()

    def get_value(self):
        if self.rank.isdigit():
            return int(self.rank)
        elif self.rank in ['Jack', 'Queen', 'King']:
            return 10
        elif self.rank == 'Ace':
            return 11
        else:
            return 0

    def __repr__(self):
        return f"{self.rank} of {self.suit}"


class Deck:
    def __init__(self, num_sets=1):
        self.num_sets = num_sets
        self.cards = self.create_deck()
        self.plastic_index = None
        self.add_plastic_card()

    def create_deck(self):
        ranks = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"]
        suits = ["Hearts", "Diamonds", "Clubs", "Spades"]
        deck = [Card(rank, suit) for _ in range(self.num_sets) for rank in ranks for suit in suits]
        return deck

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        if not self.cards:
            return None
        return self.cards.pop(0)

    def add_plastic_card(self):
        self.plastic_index = random.randint(0, len(self.cards))
        self.cards.insert(self.plastic_index, Card("Plastic", "Card"))
        
    def remove_plastic_card(self):
        self.cards.pop(self.plastic_index)
        self.plastic_index = None

    def shuffle_after_plastic_card(self):
        if self.plastic_index is not None:
            self.cards.insert(self.plastic_index, self.cards.pop(self.plastic_index))
            self.shuffle()

In [2]:
# Testing solution
# Creating a deck with 2 sets of cards
deck = Deck(num_sets=2)

# Shuffling the deck
deck.shuffle()

# Drawing cards from the deck
for _ in range(10):
    print(deck.draw_card())

Clubs of 4
Spades of 3
Diamonds of 4
Clubs of 9
Clubs of 3
Hearts of 9
Spades of 9
Diamonds of 5
Clubs of 9
Clubs of Ace


2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

In [3]:
### Creating the Hand class
class Hand:
    def __init__(self, dealer=False):
        self.dealer = dealer
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def calculate_value(self):
        total_value = sum(card.value for card in self.cards if card.rank != 'Plastic')
        num_aces = sum(1 for card in self.cards if card.rank == 'Ace')
        while total_value > 21 and num_aces:
            total_value -= 10
            num_aces -= 1
        return total_value

    def __repr__(self):
        if self.dealer:
            return "[Hidden Card] " + ", ".join(str(card) for card in self.cards[1:])
        else:
            return ", ".join(str(card) for card in self.cards)

In [4]:
## Testing the solution
# Creating a hand
hand = Hand()

# Adding some cards to the hand
card1 = Card("Hearts", "Ace")
card2 = Card("Diamonds", "5")
card3 = Card("Spades", "King")

hand.add_card(card1)
hand.add_card(card2)
hand.add_card(card3)

# Calculating and displaying the total value of the hand
total_value = hand.calculate_value()
print("Total value of the hand:", total_value)

Total value of the hand: 16


In [5]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

    def draw_card(self, deck):
        card = deck.draw_card()
        if card:
            self.hand.add_card(card)
            print(f"{self.name} drew a card:", card)
        else:
            print("No more cards in the deck.")

    def show_hand(self, dealer=False):
        print(f"{self.name}'s hand:", self.hand if not dealer else self.hand.calculate_value())

In [6]:
### Testing the solution
deck = Deck(num_sets=1)
player = Player("Alice")

# Deal initial cards to the player
for _ in range(2):
    player.draw_card(deck)

# Show player's hand
player.show_hand()

Alice drew a card: Hearts of 2
Alice drew a card: Diamonds of 2
Alice's hand: Hearts of 2, Diamonds of 2


In [7]:
class Game:
    def __init__(self, num_players):
        self.num_players = num_players
        self.deck = Deck()
        self.players = [Player(f"Player {i + 1}") for i in range(num_players)]

    def start(self):
        print("Starting the game!")
        if len(self.deck.cards) < 2 * (self.num_players + 1):
            print("Not enough cards in the deck. Reshuffling...")
            self.deck = Deck()
        for player in self.players:
            player.draw_card(self.deck)
            player.draw_card(self.deck)
        # Discard plastic card
        self.deck.cards.pop(self.deck.plastic_index)

    def play_round(self):
        print("Starting Round")
        for player in self.players:
            while True:
                player.show_hand()
                decision = input(f"{player.name}, do you want to hit or stand? (h/s): ").lower()
                if decision == 'h':
                    player.draw_card(self.deck)
                elif decision == 's':
                    break
                else:
                    print("Invalid input. Please enter 'h' to hit or 's' to stand.")

        print("Dealer's turn:")
        dealer_hand = Hand(dealer=True)
        for player in self.players:
            dealer_hand.cards.extend(player.hand.cards)
        print(f"Dealer's hand: {dealer_hand}")
        while dealer_hand.calculate_value() < 17:
            drawn_card = self.deck.draw_card()
            if drawn_card is None:
                print("No more cards in the deck.")
                break
            dealer_hand.add_card(drawn_card)
            print(f"Dealer drew a card:", dealer_hand.cards[-1])
            print(f"Dealer's hand: {dealer_hand}")
            
    def end(self):
        print("End of the Game!")
        for player in self.players:
            player.show_hand()
        print("Dealer's hand:")
        dealer_hand = Hand(dealer=True)
        for player in self.players:
            dealer_hand.cards.extend(player.hand.cards)
        print(f"Dealer's hand: {dealer_hand}")

In [8]:
# Testing the solutions
num_players = 2
game = Game(num_players)
game.start()
game.play_round()
game.end()

Starting the game!
Player 1 drew a card: Hearts of 2
Player 1 drew a card: Diamonds of 2
Player 2 drew a card: Clubs of 2
Player 2 drew a card: Spades of 2
Starting Round
Player 1's hand: Hearts of 2, Diamonds of 2


Player 1, do you want to hit or stand? (h/s):  h


Player 1 drew a card: Hearts of 3
Player 1's hand: Hearts of 2, Diamonds of 2, Hearts of 3


Player 1, do you want to hit or stand? (h/s):  s


Player 2's hand: Clubs of 2, Spades of 2


Player 2, do you want to hit or stand? (h/s):  h


Player 2 drew a card: Diamonds of 3
Player 2's hand: Clubs of 2, Spades of 2, Diamonds of 3


Player 2, do you want to hit or stand? (h/s):  s


Dealer's turn:
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Diamonds of 3
Dealer drew a card: Clubs of 3
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Diamonds of 3, Clubs of 3
Dealer drew a card: Spades of 3
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Diamonds of 3, Clubs of 3, Spades of 3
Dealer drew a card: Hearts of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Diamonds of 3, Clubs of 3, Spades of 3, Hearts of 4
Dealer drew a card: Diamonds of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Diamonds of 3, Clubs of 3, Spades of 3, Hearts of 4, Diamonds of 4
Dealer drew a card: Clubs of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Diamonds of 3, Clubs of 3, Spades of 3, Hearts of 4, Diamonds of 4, Clubs of 4
Dealer drew a card: Spades of 4
Dealer's hand: [Hidden C

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [9]:
# Testing the solutions
num_players = 2  
game = Game(num_players)

# Start the game
game.start()

# Play multiple rounds
round_count = 1
while True:
    print(f"\n--- Round {round_count} ---")
    game.play_round()
    if input("Do you want to continue playing? (y/n): ").lower() != 'y':
        break
    round_count += 1

# End the game
game.end()

Starting the game!
Player 1 drew a card: Hearts of 2
Player 1 drew a card: Diamonds of 2
Player 2 drew a card: Clubs of 2
Player 2 drew a card: Spades of 2

--- Round 1 ---
Starting Round
Player 1's hand: Hearts of 2, Diamonds of 2


Player 1, do you want to hit or stand? (h/s):  h


Player 1 drew a card: Hearts of 3
Player 1's hand: Hearts of 2, Diamonds of 2, Hearts of 3


Player 1, do you want to hit or stand? (h/s):  s


Player 2's hand: Clubs of 2, Spades of 2


...

Player 2 drew a card: Card of Plastic
Player 2's hand: Clubs of 2, Spades of 2, Card of Plastic


Player 2, do you want to hit or stand? (h/s):  s


Dealer's turn:
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Card of Plastic
Dealer drew a card: Diamonds of 3
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Card of Plastic, Diamonds of 3
Dealer drew a card: Clubs of 3
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Card of Plastic, Diamonds of 3, Clubs of 3
Dealer drew a card: Spades of 3
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Card of Plastic, Diamonds of 3, Clubs of 3, Spades of 3
Dealer drew a card: Diamonds of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Card of Plastic, Diamonds of 3, Clubs of 3, Spades of 3, Diamonds of 4
Dealer drew a card: Clubs of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Card of Plastic, Diamonds of 3, Clubs of 3, Spades of 3, Diamonds of 4, Clubs of 4
Dealer drew a card: Spades of 4

Do you want to continue playing? (y/n):  n


End of the Game!
Player 1's hand: Hearts of 2, Diamonds of 2, Hearts of 3
Player 2's hand: Clubs of 2, Spades of 2, Card of Plastic
Dealer's hand:
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Clubs of 2, Spades of 2, Card of Plastic


6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [10]:
class CardCountingPlayer(Player):
    def __init__(self, name):
        super().__init__(name)
        self.card_count = 0

    def draw_card(self, deck):
        card = deck.draw_card()
        if card:
            if card.rank.isdigit():
                value = 1 if 2 <= int(card.rank) <= 6 else (-1 if int(card.rank) >= 10 else 0)
                self.card_count += value
            elif card.rank in ['Jack', 'Queen', 'King', 'Ace']:
                self.card_count -= 1
            super().draw_card(deck)
        else:
            print("No more cards in the deck.")

    def decide(self):
        if self.card_count <= -2:
            return 'h'  # Hit if the count is very negative
        else:
            return 's'  # Stay otherwise

In [11]:
### Testing the solution
num_players = 3  

# Adding new player with card counting strategy
game.players[-1] = CardCountingPlayer("Card Counting Player")

# Start the game
game.start()

# Play multiple rounds
round_count = 1
while True:
    print(f"\n--- Round {round_count} ---")
    game.play_round()
    if input("Do you want to continue playing? (y/n): ").lower() != 'y':
        break
    round_count += 1

# End the game
game.end()

Starting the game!
Not enough cards in the deck. Reshuffling...
Player 1 drew a card: Hearts of 2
Player 1 drew a card: Diamonds of 2
Card Counting Player drew a card: Spades of 2
Card Counting Player drew a card: Diamonds of 3

--- Round 1 ---
Starting Round
Player 1's hand: Hearts of 2, Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2


Player 1, do you want to hit or stand? (h/s):  h


Player 1 drew a card: Clubs of 3
Player 1's hand: Hearts of 2, Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3


Player 1, do you want to hit or stand? (h/s):  s


Card Counting Player's hand: Spades of 2, Diamonds of 3


...

Card Counting Player drew a card: Hearts of 4
Card Counting Player's hand: Spades of 2, Diamonds of 3, Hearts of 4


Card Counting Player, do you want to hit or stand? (h/s):  s


Dealer's turn:
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3, Spades of 2, Diamonds of 3, Hearts of 4
Dealer drew a card: Diamonds of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3, Spades of 2, Diamonds of 3, Hearts of 4, Diamonds of 4
Dealer drew a card: Clubs of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3, Spades of 2, Diamonds of 3, Hearts of 4, Diamonds of 4, Clubs of 4
Dealer drew a card: Spades of 4
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3, Spades of 2, Diamonds of 3, Hearts of 4, Diamonds of 4, Clubs of 4, Spades of 4
Dealer drew a card: Hearts of 5
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3, Spades of 2, Diamonds of 3, Hearts of 4, Diamonds of 4, Clubs of 4, Spades of 4, Hearts of 5
Dealer drew a card: Diamonds of 5
De

Do you want to continue playing? (y/n):  n


End of the Game!
Player 1's hand: Hearts of 2, Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3
Card Counting Player's hand: Spades of 2, Diamonds of 3, Hearts of 4
Dealer's hand:
Dealer's hand: [Hidden Card] Diamonds of 2, Hearts of 3, Hearts of 2, Diamonds of 2, Clubs of 3, Spades of 2, Diamonds of 3, Hearts of 4


7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

In [20]:
class Dealer(Player):
    def decide(self):
        if self.hand.calculate_value() < 17:
            return 'h'  # Hit if the value of the hand is less than 17
        else:
            return 's'  # Stand otherwise


def simulate_round(self):
        round_ended = False  # Flag to track whether the round has ended
        self.deck.shuffle()  # Shuffle the deck before starting a new round
        for player in self.players:
            if isinstance(player, CardCountingPlayer):
                while player.hand.calculate_value() <= -2:
                    if self.deck.cards:  # Check if there are cards left in the deck
                        player.draw_card(self.deck)
                    else:
                        print("No more cards in the deck.")
                        round_ended = True
                        break
            elif isinstance(player, Dealer):
                while player.hand.calculate_value() < 17:
                    if self.deck.cards:  # Check if there are cards left in the deck
                        player.draw_card(self.deck)
                    else:
                        print("No more cards in the deck.")
                        round_ended = True
                        break
            else:
                decision = 's'  # Regular players always stand
                if decision == 'h':
                    if self.deck.cards:  # Check if there are cards left in the deck
                        player.draw_card(self.deck)
                    else:
                        print("No more cards in the deck.")
                        round_ended = True
        self.play_round()
        return round_ended
    
def play_scenario():
    initial_chips = 1000  # Initial number of chips for each player
    num_rounds = 50
    strategy_player_winnings = 0

    # Instantiate the game with the dealer and three regular players
    game = Game(num_players=4)

    # Add the strategy player
    strategy_player = CardCountingPlayer("Strategy Player")
    game.players[0] = strategy_player

    # Set initial chips for all players
    for player in game.players:
        player.chips = initial_chips

    # Play multiple rounds
    round_count = 0
    while round_count < num_rounds:
        print(f"\n--- Round {round_count + 1} ---")
        round_ended = simulate_round(game)
        if round_ended:
            break
        round_count += 1

    print(f"\nStrategy player's winnings after {round_count} rounds: {strategy_player_winnings}")

play_scenario()


--- Round 1 ---
Starting Round
Strategy Player's hand: 


Strategy Player, do you want to hit or stand? (h/s):  h


Strategy Player drew a card: Hearts of 8
Strategy Player's hand: Hearts of 8


Strategy Player, do you want to hit or stand? (h/s):  s


Player 2's hand: 


...

Player 4, do you want to hit or stand? (h/s):  s


Dealer's turn:
Dealer's hand: [Hidden Card] Diamonds of 6, Hearts of 3, Clubs of King
Dealer drew a card: Hearts of Ace
Dealer's hand: [Hidden Card] Diamonds of 6, Hearts of 3, Clubs of King, Hearts of Ace
Dealer drew a card: Clubs of 5
Dealer's hand: [Hidden Card] Diamonds of 6, Hearts of 3, Clubs of King, Hearts of Ace, Clubs of 5
Dealer drew a card: Spades of Jack
Dealer's hand: [Hidden Card] Diamonds of 6, Hearts of 3, Clubs of King, Hearts of Ace, Clubs of 5, Spades of Jack
Dealer drew a card: Diamonds of King
Dealer's hand: [Hidden Card] Diamonds of 6, Hearts of 3, Clubs of King, Hearts of Ace, Clubs of 5, Spades of Jack, Diamonds of King
Dealer drew a card: Diamonds of Ace
Dealer's hand: [Hidden Card] Diamonds of 6, Hearts of 3, Clubs of King, Hearts of Ace, Clubs of 5, Spades of Jack, Diamonds of King, Diamonds of Ace
Dealer drew a card: Hearts of 7
Dealer's hand: [Hidden Card] Diamonds of 6, Hearts of 3, Clubs of King, Hearts of Ace, Clubs of 5, Spades of Jack, Diamonds of Kin

Strategy Player, do you want to hit or stand? (h/s):  h


No more cards in the deck.
Strategy Player's hand: Hearts of 8


KeyboardInterrupt: Interrupted by user

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


In [17]:
import numpy as np
import matplotlib.pyplot as plt

def play_multiple_games(num_games, num_rounds_per_game):
    strategy_player_winnings_list = []

    for _ in range(num_games):
        initial_chips = 1000  # Initial number of chips for each player
        strategy_player_winnings = 0

        # Instantiate the game with the dealer and three regular players
        game = Game(num_players=4)

        # Add the strategy player
        strategy_player = CardCountingPlayer("Strategy Player")
        game.players[0] = strategy_player

        # Set initial chips for all players
        for player in game.players:
            player.chips = initial_chips

        # Play multiple rounds
        for _ in range(num_rounds_per_game):
            round_ended = game.simulate_round()
            if round_ended:
                break

        # Store strategy player's winnings at the end of the game
        strategy_player_winnings_list.append(strategy_player.chips - initial_chips)

    return strategy_player_winnings_list

def analyze_winnings(strategy_player_winnings_list):
    # Histogram the winnings
    plt.hist(strategy_player_winnings_list, bins=20, edgecolor='black')
    plt.xlabel('Strategy Player Winnings')
    plt.ylabel('Frequency')
    plt.title('Histogram of Strategy Player Winnings')
    plt.show()

    # Calculate average winnings per round
    avg_winnings_per_round = np.mean(strategy_player_winnings_list) / 50

    # Calculate standard deviation
    std_deviation = np.std(strategy_player_winnings_list)

    # Calculate probability of net winning or losing after 50 rounds
    net_win_probability = sum(1 for winnings in strategy_player_winnings_list if winnings > 0) / len(strategy_player_winnings_list)
    net_loss_probability = sum(1 for winnings in strategy_player_winnings_list if winnings < 0) / len(strategy_player_winnings_list)

    return avg_winnings_per_round, std_deviation, net_win_probability, net_loss_probability

# Main code to run 100 games of 50 rounds each
num_games = 100
num_rounds_per_game = 50
strategy_player_winnings_list = play_multiple_games(num_games, num_rounds_per_game)

# Analyze winnings
avg_winnings_per_round, std_deviation, net_win_probability, net_loss_probability = analyze_winnings(strategy_player_winnings_list)

# Print results
print("Average winnings per round:", avg_winnings_per_round)
print("Standard deviation of winnings:", std_deviation)
print("Probability of net winning after 50 rounds:", net_win_probability)
print("Probability of net losing after 50 rounds:", net_loss_probability)

AttributeError: 'Game' object has no attribute 'simulate_round'

In [ ]:
## testing the solution
scenario = TestScenario()
winnings = scenario.run_simulations(100)

9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 